<a href="https://colab.research.google.com/github/Shruti022/Healthcare-Chatbot/blob/main/Copy_of_LLM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Phase 1: Stepwise API Exploration

Step 1: Import Libraries


In [1]:
!pip install requests pandas streamlit pyngrok faiss-cpu sentence-transformers numpy

import requests
import pandas as pd
import json
import hashlib
from datetime import datetime
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 141.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%writefile build_embeddings.py
import pandas as pd
import numpy as np
import faiss
import json
from sentence_transformers import SentenceTransformer

# === REAL PATH (from readlink) ===
BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# ---------------------------------------------
# Load Data
# ---------------------------------------------
df = pd.read_csv(f"{BASE}/clinical_trials_diabetes_full.csv")

df["status"] = df["status"].astype(str).str.strip().str.title()
bad_status = ["Terminated", "Withdrawn", "Suspended", "No Longer Available", "Unknown"]
df_clean = df[~df["status"].isin(bad_status)].copy()

# ---------------------------------------------
# Chunking
# ---------------------------------------------
chunks = []
chunk_map = []

for idx, row in df_clean.iterrows():
    title = str(row.get("brief_title", "")).strip()
    summary = str(row.get("brief_summary", "")).strip()

    if len(summary) < 20:
        continue

    text = f"Title: {title}\nSummary: {summary}"
    chunks.append(text)

    chunk_map.append({
        "nct_id": row["nct_id"],
        "title": title,
        "text": text,
        "status": row["status"]
    })

print(f"Created {len(chunks)} chunks.")

# ---------------------------------------------
# Embeddings
# ---------------------------------------------
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, batch_size=64, show_progress_bar=True)

np.save(f"{BASE}/clinical_trials_diabetes_full_embeddings.npy", embeddings)
print("Saved clinical_trials_diabetes_full_embeddings.npy")

# ---------------------------------------------
# Save chunk map
# ---------------------------------------------
with open(f"{BASE}/clinical_trials_diabetes_full_chunk_map.json", "w") as f:
    json.dump(chunk_map, f)

print("Saved clinical_trials_diabetes_full_chunk_map.json")

# ---------------------------------------------
# Build & Save FAISS
# ---------------------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, f"{BASE}/clinical_trials_diabetes_full_faiss.index")

print("Saved clinical_trials_diabetes_full_faiss.index")
print("✅ Embedding build COMPLETE.")


Writing build_embeddings.py


In [4]:
!python build_embeddings.py

2025-11-24 01:18:05.578058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763947085.598073    1206 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763947085.604809    1206 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763947085.619658    1206 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763947085.619686    1206 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763947085.619690    1206 computation_placer.cc:177] computation placer alr

In [5]:
%%writefile utils.py
import json
import hashlib
from datetime import datetime

import faiss
from sentence_transformers import SentenceTransformer

# --- Confidence score from distance ---

def calculate_confidence_score(distance: float, normalization_factor: float = 1.0) -> float:
    """Inverse L2 distance score in (0,1]; closer = higher confidence."""
    return normalization_factor / (normalization_factor + float(distance))


# --- Load pre-built index + chunk map ---

def load_data_and_index(chunk_map_path: str, faiss_path: str):
    """Loads pre-built chunks and FAISS index for quick startup."""
    print("⏳ Loading pre-built RAG index...")

    with open(chunk_map_path, "r") as f:
        chunk_map = json.load(f)

    index = faiss.read_index(faiss_path)

    embed_model = SentenceTransformer("all-MiniLM-L6-v2")

    print(f"✅ RAG Index Ready: {index.ntotal} vectors loaded.")
    return embed_model, index, chunk_map


# --- Provenance logging ---

def log_provenance_step(agent_name: str, input_data, output_data, detail=None):
    """
    Creates a detailed log entry for a single agent step.
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "agent": agent_name,
        "input": input_data,
        "output": output_data,
        "detail": detail or {},
        "model_version": "gemini-2.0-flash",
    }
    return log_entry


# --- Reproducibility hash ---

def generate_reproducibility_hash(conversation_history, corpus_version: str = "v1.0"):
    """
    Generates a deterministic session hash based on the conversation history.
    """
    queries = [turn.get("query", "") for turn in conversation_history]
    raw = f"{corpus_version}|{'|'.join(queries)}"
    return hashlib.md5(raw.encode("utf-8")).hexdigest()


Writing utils.py


In [6]:
%%writefile run_bot.py
import json
import re
from typing import List, Dict, Any

import numpy as np
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

from utils import (
    load_data_and_index,
    log_provenance_step,
    generate_reproducibility_hash,
    calculate_confidence_score,
)

# ============================================================
# CONFIG / PATHS
# ============================================================
# ⚠️ FOR GITHUB: keep this as "***" and DO NOT commit your real key.
API_KEY = "***"  # <-- replace in Colab with your real key before running

if API_KEY == "***":
    print("⚠️ WARNING: You must set API_KEY in run_bot.py before running.")

genai.configure(api_key=API_KEY)
gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

#BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"
#CHUNK_PATH = f"{BASE}/clinical_trials_diabetes_full_chunk_map.json"
#FAISS_PATH = f"{BASE}/clinical_trials_diabetes_full_faiss.index"

CHUNK_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_chunk_map.json"
FAISS_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_faiss.index"

# Load embedding model, FAISS index, and chunk metadata
embed_model, faiss_index, chunk_map = load_data_and_index(CHUNK_PATH, FAISS_PATH)


# ============================================================
# AGENT 1 — Symptom Parser
# ============================================================

class SymptomParser:
    def __init__(self, model):
        self.model = model

    def parse(self, text: str):
        """
        Returns:
          parsed: dict with symptoms, duration, context, intent
          log: provenance entry
        """
        prompt = (
            "You are a medical NLP parser.\n"
            "Extract structured info and detect whether this is a greeting or a symptom query.\n\n"
            f'Input: "{text}"\n\n'
            "Return ONLY valid JSON with this format:\n"
            "{\n"
            '  "symptoms": ["list", "of", "symptoms"],\n'
            '  "duration": "text or null",\n'
            '  "context": "extra free-text context",\n'
            '  "intent": "greeting" or "symptom_query" or "other"\n'
            "}\n"
        )

        try:
            res = self.model.generate_content(prompt)
            raw = (res.text or "").strip()
            match = re.search(r"\{.*\}", raw, re.DOTALL)
            if match:
                parsed = json.loads(match.group(0))
            else:
                parsed = json.loads(raw)
        except Exception:
            # Fallback
            parsed = {
                "symptoms": [text],
                "duration": None,
                "context": "",
                "intent": "symptom_query",
            }

        log = log_provenance_step("SymptomParser", text, parsed)
        return parsed, log


# ============================================================
# AGENT 2 — ProfileAgent (simple memory)
# ============================================================

class ProfileAgent:
    def __init__(self, initial_profile: Dict[str, Any] = None):
        if initial_profile is None:
            initial_profile = {
                "user_id": "Patient",
                "conditions": ["diabetes"],
                "history": [],
            }
        self.profile = initial_profile

    def update_profile(self, turn_data: Dict[str, Any]):
        self.profile.setdefault("history", []).append(turn_data)
        snapshot = {
            "user_id": self.profile.get("user_id", "Patient"),
            "conditions": self.profile.get("conditions", []),
            "num_turns": len(self.profile["history"]),
        }
        log = log_provenance_step("ProfileAgent", turn_data, {"profile_snapshot": snapshot})
        return log


# ============================================================
# AGENT 3 — RetrievalAgent (FAISS + confidence)
# ============================================================

class RetrievalAgent:
    def __init__(self, embed_model, faiss_index, chunk_map, profile_agent: ProfileAgent = None):
        self.embed_model = embed_model
        self.index = faiss_index
        self.chunk_map = chunk_map
        self.profile_agent = profile_agent

    def retrieve(self, parsed: Dict[str, Any], top_k: int = 5):
        symptoms = parsed.get("symptoms") or []
        context = parsed.get("context") or ""
        query = (" ".join(symptoms) + " " + context).strip()

        if not query:
            retrieval = {"query": "", "trials": [], "avg_confidence": 0.0}
            log = log_provenance_step("RetrievalAgent", parsed, retrieval, {"reason": "empty_query"})
            return retrieval, log

        q_emb = self.embed_model.encode([query])
        distances, indices = self.index.search(q_emb.astype("float32"), top_k)

        trials = []
        confs = []

        for rank, idx in enumerate(indices[0]):
            item = self.chunk_map[idx]
            dist = float(distances[0][rank])
            conf = calculate_confidence_score(dist)
            confs.append(conf)

            trials.append({
                "nct_id": item["nct_id"],
                "title": item["title"],
                "text": item["text"],
                "status": item["status"],
                "distance": dist,
                "confidence": conf,
                "rank": rank + 1,
            })

        avg_conf = float(np.mean(confs)) if confs else 0.0

        retrieval = {
            "query": query,
            "trials": trials,
            "avg_confidence": avg_conf,
        }

        detail = {
            "top_k": top_k,
            "avg_confidence": avg_conf,
            "num_trials": len(trials),
        }

        log = log_provenance_step("RetrievalAgent", parsed, retrieval, detail)
        return retrieval, log


# ============================================================
# AGENT 4 — DiagnosisAdvisor (evidence-only)
# ============================================================

class DiagnosisAdvisor:
    def __init__(self, model):
        self.model = model

    def advise(self, parsed: Dict[str, Any], retrieved: Dict[str, Any]):
        trials = retrieved.get("trials", [])
        avg_conf = retrieved.get("avg_confidence", 0.0)

        # If retrieval is very low confidence, veto early
        draft = {
            "recommendation": "",
            "avg_confidence": avg_conf,
        }

        if not trials or avg_conf < 0.15:
            draft["recommendation"] = (
                "EVIDENCE IS INSUFFICIENT TO ANSWER THIS QUESTION DIRECTLY based on the "
                "retrieved clinical trials. Please consult your healthcare provider."
            )
            draft["confidence_veto"] = True
            log = log_provenance_step(
                "DiagnosisAdvisor",
                {"parsed": parsed, "retrieval_meta": {"avg_confidence": avg_conf, "num_trials": len(trials)}},
                draft,
                {"veto": True},
            )
            return draft, log

        evidence_parts = []
        for t in trials:
            evidence_parts.append(
                f"Trial {t['nct_id']} (rank {t['rank']}, confidence {t['confidence']:.2f}):\n{t['text']}\n"
            )
        evidence = "\n".join(evidence_parts)

        prompt = (
            "You are an evidence-based medical assistant summarizing clinical trials.\n"
            "You MUST answer based ONLY on the evidence below.\n"
            "If the evidence does not clearly answer the question, explicitly say:\n"
            '"EVIDENCE IS INSUFFICIENT TO ANSWER THIS QUESTION DIRECTLY."\n\n'
            "Rules:\n"
            "- Do NOT diagnose.\n"
            "- Do NOT tell the user to start/stop/change any medication.\n"
            "- Summarize what the trials studied (population, interventions, outcomes).\n"
            "- End with: 'Please discuss these findings with your healthcare provider before making any changes.'\n\n"
            "PATIENT QUERY (parsed JSON):\n"
            f"{json.dumps(parsed, indent=2)}\n\n"
            "RETRIEVED CLINICAL TRIAL EVIDENCE:\n"
            f"{evidence}\n"
        )

        try:
            res = self.model.generate_content(prompt)
            text = (res.text or "").strip()
            if not text:
                text = "EVIDENCE IS INSUFFICIENT TO ANSWER THIS QUESTION DIRECTLY."
            draft["recommendation"] = text
            draft["confidence_veto"] = False
        except Exception:
            draft["recommendation"] = "Unable to generate advice at this time."
            draft["confidence_veto"] = True

        log = log_provenance_step(
            "DiagnosisAdvisor",
            {"parsed": parsed, "retrieval_meta": {"avg_confidence": avg_conf, "num_trials": len(trials)}},
            draft,
        )
        return draft, log


# ============================================================
# AGENT 5 — ActiveSafetyFilter
# ============================================================

class ActiveSafetyFilter:
    def __init__(self, model):
        self.model = model
        self.safety_cfg = {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }

    def verify(self, advice_text: str, trials: List[Dict[str, Any]]):
        evidence_text = "\n".join(t["text"] for t in trials)

        audit_prompt = (
            "You are a Medical Safety Officer reviewing AI-generated advice.\n\n"
            "ADVICE:\n"
            f"{advice_text}\n\n"
            "EVIDENCE FROM CLINICAL TRIALS:\n"
            f"{evidence_text[:4000]}\n\n"
            "Check for safety issues:\n"
            "- If the advice suggests stopping or changing medication without a doctor → UNSAFE.\n"
            "- If it gives a diagnosis → UNSAFE.\n"
            "- If it makes claims not supported by the evidence → UNSAFE.\n\n"
            'If the advice is acceptable, respond with exactly: SAFE\n'
            'If it is not acceptable, respond starting with: CORRECTED: <safer version>\n'
        )

        try:
            res = self.model.generate_content(audit_prompt, safety_settings=self.safety_cfg)
            txt = (res.text or "").strip()
            if txt.startswith("SAFE"):
                final_text = advice_text
                status = "Pass"
            else:
                final_text = f"⚠️ SAFETY REVISION:\n{txt}"
                status = "Revised"
        except Exception:
            final_text = "⚠️ Safety filter triggered. Please consult a doctor."
            status = "Revised (API)"

        log = log_provenance_step(
            "ActiveSafetyFilter",
            {"advice": advice_text},
            {"final_text": final_text, "status": status},
        )
        return final_text, status, log


# ============================================================
# HEALTHCARE BOT (Orchestrator)
# ============================================================

class HealthcareBot:
    def __init__(self, gemini_model, embed_model, faiss_index, chunk_map, initial_profile=None):
        self.parser = SymptomParser(gemini_model)
        self.profile_agent = ProfileAgent(initial_profile)
        self.retriever = RetrievalAgent(embed_model, faiss_index, chunk_map, self.profile_agent)
        self.advisor = DiagnosisAdvisor(gemini_model)
        self.safety = ActiveSafetyFilter(gemini_model)

        self.history: List[Dict[str, Any]] = []
        self.provenance_chain: List[Dict[str, Any]] = []

    def _handle_simple_greeting(self, user_input: str):
        user_id = self.profile_agent.profile.get("user_id", "there")
        msg = (
            f"Hello {user_id}! I'm your clinical trial health assistant. "
            "Tell me your symptoms or a question about diabetes-related trials, "
            "and I’ll summarize relevant evidence. I cannot diagnose or give direct medical orders."
        )

        log = log_provenance_step(
            "GreetingAgent",
            user_input,
            msg,
            {"type": "greeting"},
        )
        self.provenance_chain.append(log)

        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])
        self.history.append({"query": user_input, "response_hash": session_hash})

        return {
            "recommendation": msg,
            "cited_trials": [],
            "safety_status": "Non-RAG",
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }

    def process_query(self, user_input: str):
        self.provenance_chain = []

        # 1. Parse
        parsed, parse_log = self.parser.parse(user_input)
        self.provenance_chain.append(parse_log)

        intent = (parsed.get("intent") or "symptom_query").lower()
        if intent == "greeting" or (not parsed.get("symptoms") and "?" not in user_input):
            return self._handle_simple_greeting(user_input)

        # 2. Retrieve
        retrieved, retrieve_log = self.retriever.retrieve(parsed)
        self.provenance_chain.append(retrieve_log)

        # 3. Advisor
        draft_advice, advise_log = self.advisor.advise(parsed, retrieved)
        self.provenance_chain.append(advise_log)

        trials = retrieved.get("trials", [])
        if draft_advice.get("confidence_veto", False) or not trials:
            final_text = draft_advice["recommendation"]
            safety_status = "Vetoed (Low Confidence)"
            evidence_list = []
        else:
            # 4. Safety
            final_text, safety_status, safety_log = self.safety.verify(
                draft_advice["recommendation"],
                trials,
            )
            self.provenance_chain.append(safety_log)
            evidence_list = trials

        nct_ids = [t["nct_id"] for t in evidence_list]

        session_hash = generate_reproducibility_hash(self.history + [{"query": user_input}])

        # 5. Update profile/history
        turn_data = {
            "query": user_input,
            "parsed": parsed,
            "nct_ids": nct_ids,
            "safety_status": safety_status,
            "session_hash": session_hash,
        }
        profile_log = self.profile_agent.update_profile(turn_data)
        self.provenance_chain.append(profile_log)
        self.history.append({"query": user_input, "response_hash": session_hash})

        return {
            "recommendation": final_text,
            "cited_trials": nct_ids,
            "safety_status": safety_status,
            "session_hash": session_hash,
            "provenance_chain": self.provenance_chain,
        }


# ============================================================
# GLOBAL BOT INSTANCE + ENTRYPOINT
# ============================================================

default_profile = {
    "user_id": "Patient",
    "conditions": ["diabetes"],
}

_bot = HealthcareBot(gemini_model, embed_model, faiss_index, chunk_map, initial_profile=default_profile)

def run_bot(user_input: str) -> Dict[str, Any]:
    return _bot.process_query(user_input)


Writing run_bot.py


UI frontend application simple web interface

https://docs.streamlit.io/develop/tutorials/chat-and-llm-apps/build-conversational-apps

In [7]:
%%writefile app.py
import streamlit as st
from run_bot import run_bot

st.title("Clinical Trial Health Advisor 🤖")

if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Get user input
if user_input := st.chat_input("Describe your symptoms or ask about trials..."):
    # User message
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Bot response
    result = run_bot(user_input)
    reply = result["recommendation"]

    with st.chat_message("assistant"):
        st.markdown(reply)

    st.session_state.messages.append({"role": "assistant", "content": reply})


Writing app.py


In [8]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

In [9]:
#AI LLM from HW4 Q3
!streamlit run app.py &>/dev/null&
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-24T01:19:00Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-24T01:19:00Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-24T01:19:03Z INF +--------------------------------------------------------------------------------------------+
2025-11-24T01:19:03Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-24T01:19:03Z INF |  https://outcomes-leader-eggs-expires.trycloudflare.co